In [10]:
# Step 1 Import necessary libraries
import os
import networkx as nx
import pickle
from node2vec import Node2Vec
import matplotlib.pyplot as plt

# Load the graph and embeddings
os.makedirs('output', exist_ok=True)
graph_path = '../output/book_translation_graph.gpickle'
with open(graph_path, 'rb') as f:
    G = pickle.load(f)


In [12]:
# Step 2 Load or input new phrases to test
test_phrases = ["Go.", "Run!", "Stop.", "Hello!"]

def preprocess_phrase(phrase):
    # Basic preprocessing steps (e.g., lowercasing, stripping whitespace)
    return phrase.strip()


In [13]:
# Step 3: Translation Function

def graph_based_translation(G, source_node, top_k=1):
    # Initialize node scores with 0, set source node to 1
    scores = {node: 0.0 for node in G.nodes}
    scores[source_node] = 1.0
    
    # Propagation loop
    for _ in range(3):
        new_scores = scores.copy()
        for node in G.nodes:
            for neighbor in G.neighbors(node):
                new_scores[neighbor] += scores[node] * G[node][neighbor].get('weight', 1.0)
        scores = new_scores

    # Extract and rank Spanish nodes
    spa_nodes = [n for n in G.nodes if G.nodes[n].get('lang') == 'SPA']
    ranked_spa_nodes = sorted(spa_nodes, key=lambda n: scores[n], reverse=True)
    
    # Return top-k translations
    return ranked_spa_nodes[:top_k]


In [14]:
# Step 4 Evaluate translations
for phrase in test_phrases:
    if phrase in G.nodes:
        translations = graph_based_translation(G, phrase, top_k=3)
        print(f"Translations for '{phrase}': {translations}")
    else:
        print(f"'{phrase}' not found in graph.")


Translations for 'Go.': ['Ve.', 'Vete.', 'Vaya.']
Translations for 'Run!': ['¡Corre!', '¡Corran!', '¡Corra!']
'Stop.' not found in graph.
Translations for 'Hello!': ['Hola.', 'Ve.', 'Vete.']


In [15]:
# Visualize embeddings or graph structure
nx.draw(G, with_labels=True)
plt.show()

In [ ]:
# Placeholder: We can compare with traditional translation models
# TODO Need to implement a comparison function with a traditional model.
